In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding, pipeline
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import warnings
warnings.filterwarnings('ignore')

In [3]:
from google.colab import files

uploaded = files.upload()

Saving amazon.csv to amazon.csv


In [4]:
df=pd.read_csv("amazon.csv")

In [5]:
# Select useful columns and drop rows with missing text or rating
df = df[['reviews.text', 'reviews.rating']].dropna()

In [6]:
# Create sentiment labels from rating
def rating_to_sentiment(r):
    if r <= 2:
        return 'negative'
    elif r == 3:
        return 'neutral'
    else:
        return 'positive'

In [7]:
df['sentiment'] = df['reviews.rating'].apply(rating_to_sentiment)

In [8]:
df_positive = df[df['sentiment'] == 'positive']
df_neutral = df[df['sentiment'] == 'neutral']
df_negative = df[df['sentiment'] == 'negative']

In [9]:
# Downsample positive class
df_positive_downsampled = resample(df_positive,
                                   replace=False,    # sample without replacement
                                   n_samples=len(df_negative),  # to match negative class count
                                   random_state=42)

In [10]:
# Combine all classes
df_balanced = pd.concat([df_positive_downsampled, df_neutral, df_negative])

In [11]:
df_balanced['sentiment'].value_counts()

,count
sentiment,
neutral,1499
positive,812
negative,812


In [12]:
# Encode labels
label_encoder = LabelEncoder()
df_balanced['label'] = label_encoder.fit_transform(df_balanced['sentiment'])

In [13]:
print("Class distribution before balancing:")
print(df_balanced['label'].value_counts())

Class distribution before balancing:
label
1    1499
2     812
0     812
Name: count, dtype: int64


In [15]:
# Balance dataset by upsampling minority classes
df_majority = df_balanced[df_balanced.label == df_balanced.label.value_counts().idxmax()]
minorities = [df_balanced[df_balanced.label == lbl] for lbl in df_balanced['label'].unique() if lbl != df_balanced.label.value_counts().idxmax()]


In [16]:
upsampled_minority_dfs = [
    resample(min_df, replace=True, n_samples=len(df_majority), random_state=42)
    for min_df in minorities
]

In [17]:
train_df_balanced = pd.concat([df_majority] + upsampled_minority_dfs).sample(frac=1, random_state=42).reset_index(drop=True)


In [18]:
print("Class distribution after balancing:")
print(train_df_balanced['label'].value_counts())

Class distribution after balancing:
label
1    1499
2    1499
0    1499
Name: count, dtype: int64


In [19]:
# Train-test split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_df_balanced['reviews.text'].tolist(),
    train_df_balanced['label'].tolist(),
    test_size=0.2,
    random_state=42
)

In [20]:
# Tokenization
checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [21]:
train_dataset = Dataset.from_dict({**train_encodings, 'label': train_labels})
val_dataset = Dataset.from_dict({**val_encodings, 'label': val_labels})

In [22]:
# Load model
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
# Metrics function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = np.argmax(predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc, 'f1': f1, 'precision': precision, 'recall': recall}

In [24]:
# Training arguments
training_args = TrainingArguments(
    # output_dir="./results",
    eval_strategy="epoch",
    save_strategy="no",
    logging_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    learning_rate=5e-5,
    weight_decay=0.01,
    report_to="none",
    load_best_model_at_end=False
)

In [25]:
# Trainer setup
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)


In [26]:
# Train and evaluate
trainer.train()
eval_results = trainer.evaluate()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.733900,0.587708,0.765556,0.762965,0.767157,0.765556
2,0.360600,0.531988,0.835556,0.833704,0.833191,0.835556
3,0.147300,0.703647,0.846667,0.844376,0.844539,0.846667


In [27]:
print("\n📊 Evaluation Results:")
for key, value in eval_results.items():
    print(f"{key}: {value:.4f}")



📊 Evaluation Results:
eval_loss: 0.7036
eval_accuracy: 0.8467
eval_f1: 0.8444
eval_precision: 0.8445
eval_recall: 0.8467
eval_runtime: 9.2084
eval_samples_per_second: 97.7370
eval_steps_per_second: 12.2710
epoch: 3.0000


In [28]:

# Save model & tokenizer
model.save_pretrained("fine-tuned-amazon-model")
tokenizer.save_pretrained("fine-tuned-amazon-model")


('fine-tuned-amazon-model/tokenizer_config.json',
 'fine-tuned-amazon-model/special_tokens_map.json',
 'fine-tuned-amazon-model/vocab.txt',
 'fine-tuned-amazon-model/added_tokens.json',
 'fine-tuned-amazon-model/tokenizer.json')

In [29]:

# Prediction pipeline
classifier = pipeline(
    "text-classification",
    model="fine-tuned-amazon-model",
    tokenizer="fine-tuned-amazon-model",
    top_k=1
)

Device set to use cuda:0


In [31]:
label_map = {f"LABEL_{i}": name for i, name in enumerate(label_encoder.classes_)}

# Test on sample Flipkart reviews or Amazon test samples
test_texts = [
    "The product arrived on time and works perfectly.",
    "Very poor quality, broke after first use.",
    "It works as expected."
]

for text in test_texts:
    result = classifier(text)[0][0]
    result["label"] = label_map[result["label"]]
    print(f"📝 {text} → {result}")

📝 The product arrived on time and works perfectly. → {'label': 'positive', 'score': 0.995431661605835}
📝 Very poor quality, broke after first use. → {'label': 'negative', 'score': 0.9973276853561401}
📝 It works as expected. → {'label': 'neutral', 'score': 0.9947375655174255}


In [37]:
import shutil
from google.colab import files
import os

# Get current directory path
current_dir = os.getcwd()

# Zip the current directory (contents inside current dir)
shutil.make_archive('current_dir_backup', 'zip', current_dir)



In [36]:
files.download('current_dir_backup.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>